<a name="top"></a>![Chisel](https://chisel.eecs.berkeley.edu/assets/img/chisel_64.png)

# Module 2.5: Putting it all Together: An FIR Filter
**Prev: [Sequential Logic](2.4_sequential_logic.ipynb)**<br>
**Next: [Generators: Parameters](3.1_parameters.ipynb)**

## Motivation
Now that you've learned the basics of Chisel, let's use that knowledge to build a FIR (finite impulse response) filter module! FIR filters are very common in digital signal processing applications. Also, the FIR filter will reappear frequently in module 3, so it's important that you don't filter out this module by skipping ahead! If you are unfamiliar with FIR filters, head over to the article on [trusty Wikipedia](https://en.wikipedia.org/wiki/Finite_impulse_response) to learn more.

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

---
# The FIR

The FIR filter you will design performs the following operation.

<img src="images/fir.jpg" width="720">

Basically, this does a elementwise multiplication of the element of the filter coefficients with the elements of the input signal and outputs the sum (also called a _convolution_).

Or, a signals definition:

$y[n] = b_0 x[n] + b_1 x[n-1] + b_2 x[n-2] + ...$
 - $y[n]$ is the output signal at time $n$
 - $x[n]$ is the input signal
 - $b_i$ are the filter coefficients or impulse response
 - $n-1$, $n-2$, ... are time $n$ delayed by 1, 2, ... cycles
 
## 8-bit Specification

Build a 4-element FIR filter where the four filter coefficients are parameters. A module skeleton and basic tests are provided for you.
Note that both the input and output are 8-bit single elements, you will need to save necessary state (like delayed signal values) using constructs like shift registers. Use the provided testers to check your implementation.
Registers with constant inputs can be created using a `ShiftRegister` of shift value 1, or by using the `RegNext` construct.

In [ ]:
class My4ElementFir(b0: Int, b1: Int, b2: Int, b3: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
  })

  ???
}

In [ ]:
// Simple sanity check: a element with all zero coefficients should always produce zero
Driver(() => new My4ElementFir(0, 0, 0, 0)) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 0)
    expect(c.io.out, 0)
    step(1)
    poke(c.io.in, 4)
    expect(c.io.out, 0)
    step(1)
    poke(c.io.in, 5)
    expect(c.io.out, 0)
    step(1)
    poke(c.io.in, 2)
    expect(c.io.out, 0)
  }
}

In [ ]:
// Simple 4-point moving average
Driver(() => new My4ElementFir(1, 1, 1, 1)) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 1)
    expect(c.io.out, 1)  // 1, 0, 0, 0
    step(1)
    poke(c.io.in, 4)
    expect(c.io.out, 5)  // 4, 1, 0, 0
    step(1)
    poke(c.io.in, 3)
    expect(c.io.out, 8)  // 3, 4, 1, 0
    step(1)
    poke(c.io.in, 2)
    expect(c.io.out, 10)  // 2, 3, 4, 1
    step(1)
    poke(c.io.in, 7)
    expect(c.io.out, 16)  // 7, 2, 3, 4
    step(1)
    poke(c.io.in, 0)
    expect(c.io.out, 12)  // 0, 7, 2, 3
  }
}

In [ ]:
// Nonsymmetric filter
Driver(() => new My4ElementFir(1, 2, 3, 4)) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 1)
    expect(c.io.out, 1)  // 1*1, 0*2, 0*3, 0*4
    step(1)
    poke(c.io.in, 4)
    expect(c.io.out, 6)  // 4*1, 1*2, 0*3, 0*4
    step(1)
    poke(c.io.in, 3)
    expect(c.io.out, 14)  // 3*1, 4*2, 1*3, 0*4
    step(1)
    poke(c.io.in, 2)
    expect(c.io.out, 24)  // 2*1, 3*2, 4*3, 1*4
    step(1)
    poke(c.io.in, 7)
    expect(c.io.out, 36)  // 7*1, 2*2, 3*3, 4*4
    step(1)
    poke(c.io.in, 0)
    expect(c.io.out, 32)  // 0*1, 7*2, 2*3, 3*4
  }
}

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val x_n1 = RegNext(io.in, 0.U)
  val x_n2 = RegNext(x_n1, 0.U)
  val x_n3 = RegNext(x_n2, 0.U)
  io.out := io.in \* b0.U(8.W) + 
    x_n1 \* b1.U(8.W) +
    x_n2 \* b2.U(8.W) + 
    x_n3 \* b3.U(8.W)
</pre></article></div></section></div>

---
# You're done!

[Return to the top.](#top)